# Pytorch로 딥러닝 제대로 배우기-중급
## Part5-2: 과대적합 & 과소적합
## 목차

1. 데이터 호출(Cifar-10)
2. 모델
3. Weight decay (L2 penalty)

### (1) 데이터 호출 (Cifar-10)

In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose

In [2]:
## Transform 지정 - ** 초급강좌에서 설명 **
transform = Compose(
    [ToTensor(),
     Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Batch size와 학습의 관계? 
batch_size = 64

trainset = datasets.CIFAR10(root='./data', train=True,
                            download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                           download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### (2) 모델

In [4]:
device = 'cuda' if torch.cuda.is_available() else "cpu"

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(32*32*3, 1024),
        nn.ReLU(),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=10, bias=True)
  )
)


### (3) 최적화(Optimization)

#### 학습/평가 함수 정의

In [5]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [6]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### SGD

#### Without penalty

In [7]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306123  [    0/50000]
loss: 2.304580  [ 6400/50000]
loss: 2.304267  [12800/50000]
loss: 2.301046  [19200/50000]
loss: 2.299531  [25600/50000]
loss: 2.298039  [32000/50000]
loss: 2.297082  [38400/50000]
loss: 2.299623  [44800/50000]
Test Error: 
 Accuracy: 18.0%, Avg loss: 2.294025 

Epoch 2
-------------------------------
loss: 2.297673  [    0/50000]
loss: 2.289077  [ 6400/50000]
loss: 2.290280  [12800/50000]
loss: 2.290241  [19200/50000]
loss: 2.295061  [25600/50000]
loss: 2.288001  [32000/50000]
loss: 2.280882  [38400/50000]
loss: 2.284498  [44800/50000]
Test Error: 
 Accuracy: 21.5%, Avg loss: 2.282909 

Epoch 3
-------------------------------
loss: 2.277173  [    0/50000]
loss: 2.278203  [ 6400/50000]
loss: 2.283442  [12800/50000]
loss: 2.273489  [19200/50000]
loss: 2.278154  [25600/50000]
loss: 2.271934  [32000/50000]
loss: 2.269907  [38400/50000]
loss: 2.275470  [44800/50000]
Test Error: 
 Accuracy: 23.9%, Avg loss: 2.267392 

Epoc

#### L2 penalty 0.05

In [9]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=0.05)

In [10]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297459  [    0/50000]
loss: 2.303988  [ 6400/50000]
loss: 2.300615  [12800/50000]
loss: 2.303490  [19200/50000]
loss: 2.297089  [25600/50000]
loss: 2.298991  [32000/50000]
loss: 2.307040  [38400/50000]
loss: 2.290476  [44800/50000]
Test Error: 
 Accuracy: 18.7%, Avg loss: 2.294098 

Epoch 2
-------------------------------
loss: 2.294734  [    0/50000]
loss: 2.284414  [ 6400/50000]
loss: 2.290050  [12800/50000]
loss: 2.285789  [19200/50000]
loss: 2.290973  [25600/50000]
loss: 2.288524  [32000/50000]
loss: 2.293942  [38400/50000]
loss: 2.293323  [44800/50000]
Test Error: 
 Accuracy: 19.6%, Avg loss: 2.286909 

Epoch 3
-------------------------------
loss: 2.292247  [    0/50000]
loss: 2.286367  [ 6400/50000]
loss: 2.288236  [12800/50000]
loss: 2.292526  [19200/50000]
loss: 2.282960  [25600/50000]
loss: 2.286618  [32000/50000]
loss: 2.278522  [38400/50000]
loss: 2.281231  [44800/50000]
Test Error: 
 Accuracy: 20.1%, Avg loss: 2.279841 

Epoc

#### L2 penalty 0.1

In [11]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=0.1)

In [12]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300839  [    0/50000]
loss: 2.295878  [ 6400/50000]
loss: 2.302687  [12800/50000]
loss: 2.294059  [19200/50000]
loss: 2.296706  [25600/50000]
loss: 2.302140  [32000/50000]
loss: 2.290491  [38400/50000]
loss: 2.291849  [44800/50000]
Test Error: 
 Accuracy: 13.8%, Avg loss: 2.297049 

Epoch 2
-------------------------------
loss: 2.301096  [    0/50000]
loss: 2.295632  [ 6400/50000]
loss: 2.293118  [12800/50000]
loss: 2.298259  [19200/50000]
loss: 2.290100  [25600/50000]
loss: 2.300769  [32000/50000]
loss: 2.294082  [38400/50000]
loss: 2.290328  [44800/50000]
Test Error: 
 Accuracy: 17.2%, Avg loss: 2.293368 

Epoch 3
-------------------------------
loss: 2.289472  [    0/50000]
loss: 2.292726  [ 6400/50000]
loss: 2.294420  [12800/50000]
loss: 2.295885  [19200/50000]
loss: 2.291976  [25600/50000]
loss: 2.296063  [32000/50000]
loss: 2.288407  [38400/50000]
loss: 2.290373  [44800/50000]
Test Error: 
 Accuracy: 18.3%, Avg loss: 2.291479 

Epoc

#### L2 penalty 0.9

In [13]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=0.9)

In [14]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300223  [    0/50000]
loss: 2.305613  [ 6400/50000]
loss: 2.296751  [12800/50000]
loss: 2.305841  [19200/50000]
loss: 2.306286  [25600/50000]
loss: 2.302212  [32000/50000]
loss: 2.298262  [38400/50000]
loss: 2.302646  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302369 

Epoch 2
-------------------------------
loss: 2.302385  [    0/50000]
loss: 2.303618  [ 6400/50000]
loss: 2.301514  [12800/50000]
loss: 2.302495  [19200/50000]
loss: 2.303147  [25600/50000]
loss: 2.301382  [32000/50000]
loss: 2.301815  [38400/50000]
loss: 2.302889  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302586 

Epoch 3
-------------------------------
loss: 2.301123  [    0/50000]
loss: 2.303142  [ 6400/50000]
loss: 2.302218  [12800/50000]
loss: 2.302373  [19200/50000]
loss: 2.302278  [25600/50000]
loss: 2.302993  [32000/50000]
loss: 2.303942  [38400/50000]
loss: 2.302426  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302589 

Epoc

#### L2 penalty 10.0

In [14]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=10.0)

In [ ]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

#### Adam

#### Without Penalty

In [15]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [16]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304082  [    0/50000]
loss: 1.933879  [ 6400/50000]
loss: 1.562420  [12800/50000]
loss: 1.566714  [19200/50000]
loss: 1.500309  [25600/50000]
loss: 1.627737  [32000/50000]
loss: 1.531193  [38400/50000]
loss: 1.566987  [44800/50000]
Test Error: 
 Accuracy: 46.5%, Avg loss: 1.508088 

Epoch 2
-------------------------------
loss: 1.532550  [    0/50000]
loss: 1.383837  [ 6400/50000]
loss: 1.712213  [12800/50000]
loss: 1.574761  [19200/50000]
loss: 1.340460  [25600/50000]
loss: 1.444560  [32000/50000]
loss: 1.309289  [38400/50000]
loss: 1.590051  [44800/50000]
Test Error: 
 Accuracy: 48.9%, Avg loss: 1.464916 

Epoch 3
-------------------------------
loss: 1.193930  [    0/50000]
loss: 1.287709  [ 6400/50000]
loss: 1.531192  [12800/50000]
loss: 1.311063  [19200/50000]
loss: 1.206421  [25600/50000]
loss: 1.456125  [32000/50000]
loss: 1.197367  [38400/50000]
loss: 1.282072  [44800/50000]
Test Error: 
 Accuracy: 50.6%, Avg loss: 1.407399 

Epoc

#### L2 Penalty 0.1

In [17]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1)

In [18]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300364  [    0/50000]
loss: 2.299894  [ 6400/50000]
loss: 2.303112  [12800/50000]
loss: 2.304375  [19200/50000]
loss: 2.302946  [25600/50000]
loss: 2.301346  [32000/50000]
loss: 2.301550  [38400/50000]
loss: 2.303118  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302603 

Epoch 2
-------------------------------
loss: 2.302287  [    0/50000]
loss: 2.302473  [ 6400/50000]
loss: 2.304239  [12800/50000]
loss: 2.302402  [19200/50000]
loss: 2.301298  [25600/50000]
loss: 2.302886  [32000/50000]
loss: 2.301692  [38400/50000]
loss: 2.303314  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302641 

Epoch 3
-------------------------------
loss: 2.302192  [    0/50000]
loss: 2.303226  [ 6400/50000]
loss: 2.303184  [12800/50000]
loss: 2.302469  [19200/50000]
loss: 2.302809  [25600/50000]
loss: 2.304734  [32000/50000]
loss: 2.301352  [38400/50000]
loss: 2.302637  [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302603 

Epoc

#### L2 Penalty 0.001

In [19]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.001)

In [20]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303187  [    0/50000]
loss: 1.780736  [ 6400/50000]
loss: 1.518081  [12800/50000]
loss: 1.640464  [19200/50000]
loss: 1.768692  [25600/50000]
loss: 1.295267  [32000/50000]
loss: 1.532015  [38400/50000]
loss: 1.565418  [44800/50000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 1.611962 

Epoch 2
-------------------------------
loss: 1.681200  [    0/50000]
loss: 1.437905  [ 6400/50000]
loss: 1.691004  [12800/50000]
loss: 1.618219  [19200/50000]
loss: 1.680746  [25600/50000]
loss: 1.442283  [32000/50000]
loss: 1.559177  [38400/50000]
loss: 1.536353  [44800/50000]
Test Error: 
 Accuracy: 44.6%, Avg loss: 1.548827 

Epoch 3
-------------------------------
loss: 1.479528  [    0/50000]
loss: 1.620764  [ 6400/50000]
loss: 1.327159  [12800/50000]
loss: 1.557962  [19200/50000]
loss: 1.397873  [25600/50000]
loss: 1.793994  [32000/50000]
loss: 1.414264  [38400/50000]
loss: 1.565879  [44800/50000]
Test Error: 
 Accuracy: 47.2%, Avg loss: 1.476799 

Epoc